# 구조적 스트리밍의 기초
- 구조적 스트리밍을 구성하는 핵심 개념과 구조적 스트리밍이 얼마나 사용하기 쉬운지 살펴볼것임

## 구조적 스트리밍의 기초
<img src ="https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png" width=70% />
- 스파크의 <strong>구조적 API</strong>(DataFrame, Dataset, SQL)를 사용함


- 스트림 데이터를 <strong>데이터가 계속해서 추가되는 테이블</strong>처럼 다룬다는 것이 핵심 아이디어


- 스트리밍 잡은 <strong>계속해서 신규 입력 데이터를 확인 및 처리</strong>함


- <strong>내고장성을 보장</strong>하면서 신규 데이터가 유입될 때마다 <strong>효율</strong>적으로 처리 결과 갱신
  - 스트림 처리와 관련된 쿼리 구문이 따로 있지 않음(배치와 동일)
    - 그냥 쿼리 실행 유형만 스트리밍으로 지정하면 됨
    - 일부 제약은 있음
  - 내부적으로 사용자의 쿼리를 어떻게 증분할지 자동으로 파악함
  
  
- 스파크의 다른 기능들과 통합해서 데이터에 실시간으로 반응하는 <strong>통합 빅데이터 처리 애플리케이션</strong>을 만들 수 있음
  - 이로 인해 <strong>하나의 프레임워크</strong>를 이용해서 전체 애플리케이션을 쉽게 정의하고 <strong>여러 처리 영역에서 일관된 결과</strong>를 얻을 수 있도록 함
  - ex) 사용자가 스파크 SQL을 통해 대화형으로 조회하는 테이블을 계속해서 갱신
  - ex) MLlib을 사용해 학습한 머신러닝 모델을 적용한 결과를 제공
  - ex) 스파크 데이터소스의 오프라인 데이터와 스트림을 조인

## 핵심 개념

### 트랜스포메이션과 액션
- <strong>기존 트랜스포메이션 개념과 거의 같음</strong>

- 근데 몇 가지 제약사항은 있음
  - 엔진에서 증분 처리를 할 수 없는 일부 쿼리 유형

- 액션은 단 한 가지
  - 스트림 처리를 시작한 뒤 <strong>연속적으로 처리해 결과를 출력하는 액션만</strong> 있음

### 입력 소스
- 스트리밍 방식으로 데이터를 읽을 수 있는 몇 가지 입력 소스(버전 3.1)
  - HDFS나 S3 등 분산 파일 시스템의 파일
  - 카프카
  - 테스팅용 소켓 소스
  - 테스팅용 레이트 소스
    - 특정 개수의 row를 매초마다 생성

### 싱크
- 싱크로 스트림의 <strong>결과를 저장할 목적지</strong>를 명시함

- 싱크와 실행 엔진은 데이터 처리의 진행 상황을 신뢰도 있고 정확하게 추적하는 역할을 함

- 지원하는 출력용 싱크(버전 3.1)
  - 거의 모든 파일 포맷
  - 카프카
  - 출력 레코드에 임의 연산을 실행하는 foreach 싱크
  - 디버깅용 콘솔 싱크
  - 디버깅용 메모리 싱크

### 출력 모드
- 싱크를 정의하기 위해서는 데이터를 출력하는 방법도 정의해야함

- 지원하는 출력 모드(버전 3.1)
  - append: 싱크에 신규 레코드만 추가(default)
  - update: 변경 대상 레코드 자체를 갱신
  - complete: 전체 출력 내용 재작성
  
- 특정 쿼리와 싱크는 일부 출력모드만 지원함
  - ex) 스트림에 map연산만 하는 잡
    - 매번 전체 데이터를 신규 파일로 저장하는 complete 모드는 적합하지 않음

### 트리거
- 트리거는 <strong>데이터 출력 시점</strong>을 정의함
  - 언제 신규 데이터를 확인하고 결과를 갱신할지 정의함

- <strong>기본적으로는 마지막 입력 데이터를 처리한 직후</strong>에 신규 데이터를 확인하고 결과를 갱신함
  - 근데 파일 싱크를 사용하는 경우, 작은 크기의 파일이 여러 개 생길 수 있음

- 처리 시간(고정된 주기로만 신규 데이터 탐색)기반의 트리거도 지원

### 이벤트 시간 처리
- 이벤트 시간 기준의 처리도 지원함

- 이 처리 방식은 무작위로 도착한 <strong>레코드 내부에 기록된 타임스탬프</strong>를 기준으로함

- 두 가지 핵심 아이디어
  - <strong>이벤트 시간 데이터</strong>
    - 이벤트 시간은 데이터에 기록된 시간 필드를 의미
    - 표준 SQL 연산자를 이용해 그룹화, 집계, 윈도우 처리를 하여 이벤트 시간 필드를 인식하도록 해서 이벤트 시간 처리 가능
    
  - <strong>워터마크</strong>
    - 시간 제한을 설정할 수 있는 스트리밍 시스템 기능
    - 늦게 들어온 이벤트를 어디까지 처리할지 시간을 제한할 수 있음

## 구조적 스트리밍 활용
- 데이터셋: 인간 행동 인지를 위한 이기종(Heterogeneity Human Activity Recognition) 데이터셋
  - 스마트폰, 스마트워치 사용자가 자전거 타기, 앉기, 일어서기, 걷기 등의 활동을 하는 동안 기록된 센서 데이터로 구성
    - 기기는 여러 종류로 구성
    - 참여자는 9명

In [0]:
path = 'FileStore/tables/bin/activity-data'

### 정적 Dataframe

In [0]:
static = spark.read.json(path)
dataSchema = static.schema

In [0]:
dataSchema

Out[ 16 ]: StructType(List(StructField(Arrival_Time,LongType,true),StructField(Creation_Time,LongType,true),StructField(Device,StringType,true),StructField(Index,LongType,true),StructField(Model,StringType,true),StructField(User,StringType,true),StructField(gt,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(z,DoubleType,true)))

In [0]:
display(static.take(5))

Arrival_Time,Creation_Time,Device,Index,Model,User,gt,x,y,z
1424686735090,1424686733090638193,nexus4_1,18,nexus4,g,stand,3.356934E-4,-5.645752E-4,-0.018814087
1424686735292,1424688581345918092,nexus4_2,66,nexus4,g,stand,-0.005722046,0.029083252,0.005569458
1424686735500,1424686733498505625,nexus4_1,99,nexus4,g,stand,0.0078125,-0.017654419,0.010025024
1424686735691,1424688581745026978,nexus4_2,145,nexus4,g,stand,-3.814697E-4,0.0184021,-0.013656616
1424686735890,1424688581945252808,nexus4_2,185,nexus4,g,stand,-3.814697E-4,-0.031799316,-0.00831604


----
- 타임스탬프, 장비 및 모델 정보, 사용자 정보가 있음
- gt필드는 해당 시점의 사용자 행동 유형임

### 스트리밍 Dataframe

In [0]:
#read 대신 readStream 메서드
streaming = spark.readStream.schema(dataSchema).option('maxFilesPerTrigger',1).json('/'+path)

In [0]:
#트랜스포메이션 정의
activityCounts = streaming.groupBy('gt').count()

In [0]:
#액션 정의
#쿼리 결과를 내보낼 목적지나 싱크를 지정해야함

activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format('memory').outputMode('complete').start()#.awaitTermination()

----
- 위 예제에서는 스트림 처리에 사용하는 쿼리의 이름을 'activity_counts'로 정의함
- 결과를 메모리에 저장하는 <strong>메모리 싱크</strong>를 사용함
- 출력모드는 트리거를 실행한 후 모든 키와 데이터 수를 다시 저장하는 <strong>complete</strong> 출력 모드 사용
- 운영용 애플리케이션에서는 반드시 <strong>awaitTermination()</strong>을 지정해야함
  - 실행 중에 <strong>드라이버 프로세스가 종료되는 상황을 막아줌</strong>

In [0]:
#실행중인 스트림 목록 확인
spark.streams.active

Out[ 36 ]: [<pyspark.sql.streaming.StreamingQuery at 0x7f9002e98320>]

#### 결과 조회

In [0]:
from time import sleep

#스트리밍 쿼리의 결과를 1초마다 출력
for _ in range(3):
  spark.sql('select * from activity_counts').show()
  sleep(1)

+----------+------+
 gt| count|
+----------+------+
 stairsup|282351|
 sit|332342|
 stand|307391|
 walk|357886|
 bike|291500|
stairsdown|252810|
 null|282056|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|292806|
 sit|344652|
 stand|318775|
 walk|371141|
 bike|302296|
stairsdown|262173|
 null|292504|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|292806|
 sit|344652|
 stand|318775|
 walk|371141|
 bike|302296|
stairsdown|262173|
 null|292504|
+----------+------+

## 스트림 트랜스포메이션

### 선택과 필터링
- 구조적 스트리밍은 DataFrame의 모든 함수와 개별 컬럼을 처리하는 선택과 필터링, 단순 트랜스포메이션을 지원함

In [0]:
from pyspark.sql import functions as F
simpleTransform = streaming.withColumn('stairs', F.expr("gt like '%stairs%'"))\
.where('stairs')\
.where('gt is not null')\
.select('gt', 'model', 'arrival_time', 'creation_time')\
.writeStream.queryName('simple_transform')\
.format('memory')\
.outputMode('append').start()

In [0]:
#이제 두 개의 스트림이 실행 중
spark.streams.active

Out[ 41 ]: [<pyspark.sql.streaming.StreamingQuery at 0x7f9002e1e940>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f9002e1e7f0>]

#### 결과 조회

In [0]:
spark.sql('select distinct(gt) from simple_transform').show()

+----------+
 gt|
+----------+
 stairsup|
stairsdown|
+----------+

In [0]:
for _ in range(3):
  spark.sql('select * from simple_transform').limit(3).show()
  sleep(1)

+----------+------+-------------+-------------------+
 gt| model| arrival_time| creation_time|
+----------+------+-------------+-------------------+
stairsdown|nexus4|1424788789523|1424788793003531175|
stairsdown|nexus4|1424788789721|1424790635038301111|
 stairsup|nexus4|1424687983665|1424689829720400333|
+----------+------+-------------+-------------------+

+--------+------+-------------+-------------------+
 gt| model| arrival_time| creation_time|
+--------+------+-------------+-------------------+
stairsup|nexus4|1424687983725|1424687981731838118|
stairsup|nexus4|1424687984021|1424687982029994857|
stairsup|nexus4|1424687984421|1424687982426655964|
+--------+------+-------------+-------------------+

+--------+------+-------------+-------------------+
 gt| model| arrival_time| creation_time|
+--------+------+-------------+-------------------+
stairsup|nexus4|1424687983631|1424687981636012923|
stairsup|nexus4|1424687983840|1424687981842616927|
stairsup|nexus4|1424687984229|1424687982235378157|
+--------+------+-------------+-------------------+

### 집계
- 구조적 API처럼 임의의 집계 연산을 지정할 수 있음

In [0]:
deviceModelStats = streaming.cube('gt', 'model').avg()\
.drop("avg(Arrival_Time)")\
.drop("avg(Creation_Time)")\
.drop('avg(Index)')\
.writeStream.queryName('device_counts').format('memory')\
.outputMode('complete').start()

In [0]:
spark.streams.active

Out[ 47 ]: [<pyspark.sql.streaming.StreamingQuery at 0x7f9002e3ff60>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f9002e3f860>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f9002e3f6a0>]

#### 결과 조회

In [0]:
for _ in range(3):
  spark.sql('select * from device_counts').limit(5).show()
  sleep(1)

+----------+------+--------------------+--------------------+--------------------+
 gt| model| avg(x)| avg(y)| avg(z)|
+----------+------+--------------------+--------------------+--------------------+
 null| null|4.796918779024285...|-0.00601540958963...|-0.01013356489164...|
stairsdown| null|0.021613908669165332|-0.03249018824752...| 0.12035922691504056|
stairsdown|nexus4|0.021613908669165335|-0.03249018824752...| 0.12035922691504056|
 stand| null|-3.11082189691728E-4|3.218461665975322E-4|2.141300040636463...|
 sit| null|-5.49433244039590...|2.791446281700071...|-2.33994461689892...|
+----------+------+--------------------+--------------------+--------------------+

+-----+------+--------------------+--------------------+--------------------+
 gt| model| avg(x)| avg(y)| avg(z)|
+-----+------+--------------------+--------------------+--------------------+
stand| null|-3.11082189691728E-4|3.218461665975322E-4|2.141300040636463...|
 sit| null|-5.49433244039590...|2.791446281700071...|-2.33994461689892...|
 walk|nexus4|-0.00390116006094368|0.001052508689953...|-6.95435553042998...|
 null|nexus4|-0.00847688860109...|-7.30455258739178...|0.003090601491419...|
 bike|nexus4|0.022688759550866824|-0.00877912156368...| -0.0825100166341237|
+-----+------+--------------------+--------------------+--------------------+

+-----+------+--------------------+--------------------+--------------------+
 gt| model| avg(x)| avg(y)| avg(z)|
+-----+------+--------------------+--------------------+--------------------+
 null|nexus4|4.796918779024285...|-0.00601540958963...|-0.01013356489164...|
stand|nexus4|-3.11082189691728E-4|3.218461665975322E-4|2.141300040636463...|
 walk| null|-0.00390116006094368|0.001052508689953...|-6.95435553042998...|
stand| null|-3.11082189691728E-4|3.218461665975322E-4|2.141300040636463...|
 null| null|4.796918779024285...|-0.00601540958963...|-0.01013356489164...|
+-----+------+--------------------+--------------------+--------------------+

### 조인

In [0]:
historicalAgg = static.groupBy('gt', 'model').avg()
deviceModelStats = streaming.drop('Arrival_Time', 'Creation_Time','Index')\
.cube('gt', 'model').avg()\
.join(historicalAgg, ['gt','model'])\
.writeStream.queryName('device_counts2').format('memory')\
.outputMode('complete').start()

In [0]:
spark.streams.active

Out[ 57 ]: [<pyspark.sql.streaming.StreamingQuery at 0x7f90025ffbe0>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f90025ffa58>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f90025ff9b0>,
 <pyspark.sql.streaming.StreamingQuery at 0x7f90025ff978>]

#### 결과 조회

In [0]:
for _ in range(3):
  spark.sql('select * from device_counts2').show()
  sleep(1)

+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
 gt| model| avg(x)| avg(y)| avg(z)| avg(Arrival_Time)| avg(Creation_Time)| avg(Index)| avg(x)| avg(y)| avg(z)|
+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
 bike|nexus4|0.021949482812608244|-0.00749751739872...|-0.08270261150639441|1.424751134339993...|1.424752127369581...| 326459.6867328154|0.022688759550866883|-0.00877912156368...|-0.08251001663412375|
 walk|nexus4|-0.00285732139643...|0.003373111639646...|-0.00155088920916...|1.424746420641789...|1.424747351060678...|149760.09974990616|-0.00390116006094...|0.001052508689953...|-6.95435553042992...|
stairsdown|nexus4| 0.02262820081168685|-0.03079508201793...| 0.12088580480524011|1.424744591412858...|1.424745503635642...|230452.44623187225| 0.02161390866916532|-0.03249018824752...| 0.1203592269150406|
 sit|nexus4|-6.44713381746548...|2.614244124546442...|-1.55316137749796...|1.424741207868232...|1.424742112220356...| 74577.84690275553|-5.49433244039586...|2.791446281700071E-4|-2.33994461689889...|
 stand|nexus4|-3.89372639367013...|3.238874567024347...|2.323141750435506E-4|1.424743637921212...|1.424744579547465...|31317.877585550017|-3.11082189691723...| 3.21846166597532E-4|2.141300040636485...|
 null|nexus4|-0.00981882283597...|-0.00189753526302...|0.003903477859879...|1.424749002876348E12|1.424749919482132...| 219276.9663669269|-0.00847688860109...|-7.30455258739180...|0.003090601491419937|
 stairsup|nexus4|-0.02424235941716...|-0.00542728576322...|-0.10011072208887009|1.424745996101166E12|1.424746915892742...|227912.96550673083|-0.02479965287771...|-0.00800392344379...| -0.100340884150604|
+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+

+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
 gt| model| avg(x)| avg(y)| avg(z)| avg(Arrival_Time)| avg(Creation_Time)| avg(Index)| avg(x)| avg(y)| avg(z)|
+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
 bike|nexus4|0.021949482812608244|-0.00749751739872...|-0.08270261150639441|1.424751134339993...|1.424752127369581...| 326459.6867328154|0.022688759550866883|-0.00877912156368...|-0.08251001663412375|
 walk|nexus4|-0.00285732139643...|0.003373111639646...|-0.00155088920916...|1.424746420641789...|1.424747351060678...|149760.09974990616|-0.00390116006094...|0.001052508689953...|-6.95435553042992...|
stairsdown|nexus4| 0.02262820081168685|-0.03079508201793...| 0.12088580480524011|1.424744591412858...|1.424745503635642...|230452.44623187225| 0.02161390866916532|-0.03249018824752...| 0.1203592269150406|
 sit|nexus4|-6.44713381746548...|2.614244124546442...|-1.55316137749796...|1.424741207868232...|1.424742112220356...| 74577.84690275553|-5.49433244039586...|2.791446281700071E-4|-2.33994461689889...|
 stand|nexus4|-3.89372639367013...|3.238874567024347...|2.323141750435506E-4|1.424743637921212...|1.424744579547465...|31317.877585550017|-3.11082189691723...| 3.21846166597532E-4|2.141300040636485...|
 null|nexus4|-0.00981882283597...|-0.00189753526302...|0.003903477859879...|1.424749002876348E12|1.424749919482132...| 219276.9663669269|-0.00847688860109...|-7.30455258739180...|0.003090601491419937|
 stairsup|nexus4|-0.02424235941716...|-0.00542728576322...|-0.10011072208887009|1.424745996101166E12|1.424746915892742